This script uses Selenium to scrape images from Pinterest based on a specific search query, downloads them, and saves them locally. It navigates through the page, continuously scrolling to load more images and downloads a specified number of images.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
import os
 
URL = "https://ru.pinterest.com/search/pins/?q=harry%20potter%20drawings%20people&rs=typed"
SAVE_FOLDER = "images"
MAX_IMAGES = 1000

os.makedirs(SAVE_FOLDER, exist_ok=True)

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get(URL)

# Wait until the page is fully loaded
wait = WebDriverWait(driver, 10)

# Function to scroll the page down
def scroll_down():
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

downloaded = 0
image_urls = set()

while downloaded < MAX_IMAGES:
    images = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))

    for img in images:
        if downloaded >= MAX_IMAGES:
            break

        try:
            src = img.get_attribute("src")
            if src and src not in image_urls:
                image_urls.add(src)
                response = requests.get(src)
                
                if response.status_code == 200:
                    with open(os.path.join(SAVE_FOLDER, f"{downloaded + 1}.jpg"), "wb") as f:
                        f.write(response.content)
                    downloaded += 1
                    print(f"Downloaded: {downloaded} / {MAX_IMAGES}")
        
        except Exception as e:
            print(f"Error while downloading: {e}")

    scroll_down()

print("Finished! Images downloaded.")
driver.quit()
